In [1]:
'''
#Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
callbacks=[print_callback])

Using TensorFlow backend.


corpus length: 600893
total chars: 57
nb sequences: 200285
Vectorization...
Build model...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/60
200285/200285 [==============================] - 175s 872us/step - loss: 1.9850

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "eged and
autocratic--likewise a second a"
eged and
autocratic--likewise a second and of the sals of the proprous of the propers of the deserves of the same and one of the same of the sension of the decelt of the man and one man of the same of the san and the propled and the same and the same of the sacrest of the man the paspent of the same the same of the reperiods of the propless and resens of the believer of the same of the man in the same of the same and believe and san and
----- diversity: 0.5
----- Generating with seed: "eged and
autocratic--likewise a second a"
eged and
autocratic--likewise a sec

that investigate a the good and his freedom, and perhaps to be something of his constitute the same and the become an and the belogical of the fless of the more extent of the an and of the deal experion of a conduct of the soul subtlerknons in their deader, and understand the former and example the inductive the antire of those and in the constinct and imperfors here the prost a spire by more devent is profess
an
----- diversity: 1.0
----- Generating with seed: "ive duty of any science
that investigate"
ive duty of any science
that investigateness in theill himself time, those undominating
throughtitt, a paritistbus
flect show are a mists-pond shums
bethomres and wholly honeker is is command, itself--and that been and futute, the entulies 3"men memhlate in fast casing and
adding women to some greater adout of same
in all full every not comean an its standers
eap, well
a plaman yet if thereforefulness and spirit concept that thus spire 
----- diversity: 1.2
----- Generating with seed: "

accedical timple from--callly learts, alwais in the condempolont insouric into hi
Epoch 8/60
200285/200285 [==============================] - 181s 902us/step - loss: 1.4214

----- Generating text after Epoch: 7
----- diversity: 0.2
----- Generating with seed: "e recovers it from among the ashes.

84."
e recovers it from among the ashes.

84. the something the order to the sense of the former of the superstition of the something the the self-possible and and and and the life and and and the more with the former the problem, and the highest to the fact that the seems to the demand the former the the and and and and the sense of the such a manifold the consider of the superstition of the most such a man is a problem of the consider of t
----- diversity: 0.5
----- Generating with seed: "e recovers it from among the ashes.

84."
e recovers it from among the ashes.

84. the best honours. the logical from the from the people and an artion of the love of the gransical represent in the serve do

/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:67: RuntimeWarning: divide by zero encountered in log


...

a sensible of the same more contrary, the more and as a sensible of the consider and contrary, and a philosophers and as the morality of the same conscience of the standpoint of the sensible of the same more conscience of the same more and the soul, in the extraordinar of the same conscience of the same more and as the sensible of
----- diversity: 0.5
----- Generating with seed: "n referred to is a just one. at the same"
n referred to is a just one. at the same powerful the kind of the philosophers and stated, morals of the conscience is the sensible
conscience of which development, it demand of the considers of an exception and substanding of the extraordinar the soul of the procle, and always extraordinarisoneralists of the master and made of the exception, the contrary, is a disent their feeling, even in the conscience of the appropree of the histori
----- diversity: 1.0
----- Generating with seed: "n referred to is a just one. at the same"
n referred to is a just one. at the same;


accurate, there is no right that can condiginvees could neceap the eless--the will still reforedress--that there is cure direction" renating granits of forces or to philosophics justicesmes. that parcession; not always liitnerly, acts, testips xpreitance
well-reriately.
     might dalfaxion is! there jawaves (that should interpration to self view of
the neither did at peofoptifical intelligiances,
amount whether benessrines. thus loves
Epoch 47/60
200285/200285 [==============================] - 131s 656us/step - loss: 1.3264

----- Generating text after Epoch: 46
----- diversity: 0.2
----- Generating with seed: "must first show whether it can exercise
"
must first show whether it can exercise
the same as the subject of the most subtlety of the problem of the subject of the subject the same problem of the same degrees of the sublime and the problem of the problem of the subject of the same as the problem of the subject the soul only the most assume the subject the same as the problem o

we, whose duty is wakefule will about himself which for the master the spirity, the condition and the ascetic musiciar and interpretation of the soul for a counterial and action of the conception of a
spiritn, and in the world for reputice the conceptions, and all fact the matter the spiritual promisively interpretation of an
instinct of a counter in the soul and distarily in the sees and interpretation of the sleed
which
----- diversity: 1.0
----- Generating with seed: "lthier--sleep,
we, whose duty is wakeful"
lthier--sleep,
we, whose duty is wakefuls its betray as
ohecting only the resations which incivel, one in setest-""small just through enviddern self-religious en, 
as
farr" to be dirady in the magavous,
faith, look the fact that here are sonical explained exact soavally and
prificantalation in erevid, paridation", in a such
which
men
wage of their
south indeeding whole recognitatious will then sympathy, i an
and founder unstandaven the 
----- diversity: 1.2
----- Generating wit

deker dominakqre to tepii atsed to them owing to the uncessificated, bid pridestrwiaten: he shoet case, "from it concerning powers nor enough.
powes world to the
fit much has
'more
Epoch 54/60
200285/200285 [==============================] - 130s 649us/step - loss: 1.3251

----- Generating text after Epoch: 53
----- diversity: 0.2
----- Generating with seed: "ould seem to be a rendezvous of question"
ould seem to be a rendezvous of questions and as a state of the presently and the problem of the presently and as the process and men with the same problem of the same problem of the same and states and all the present and signfficianion of the presently and still and the problem of the most artists and interpreted to the same process and with all the process and as the presently and supportent and in the problem of the presently and as
----- diversity: 0.5
----- Generating with seed: "ould seem to be a rendezvous of question"
ould seem to be a rendezvous of question, preferences and exces

delight--is not a proceam".--and notly a call old, in old to fo
churcheds at the los that--yet circlen this congurustious actsitic will from and as so, i cannot
please the time, the fundartical latter world, swinces of denymar
of our needings elsies standsome if virtue-from its tyen party of him rowings as even himself, that living only short! and conspeciation of this of the first
charment by natur
----- diversity: 1.2
----- Generating with seed: "g out, a closing of windows, an inner
de"
g out, a closing of windows, an inner
defuting mest is right: when there and as foredlaw"--as
all verped gratifully terable, world
to
doyfwlle elacimiusly
live "cusious pordantuous ideals, there southers. igary
body;as,
if botcces a humanfulness" is mady to delicate, it is overs of his printion of carte same such   where hogow so mucders amtation to enciption
exultion of most, rathew--suffative civilization andses--the intrees than tymi
Epoch 58/60
200285/200285 [==============================] - 133